In [60]:
import ltn
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- 1. CONFIGURAÇÃO ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Rodando em: {device}")

# --- 2. GERAÇÃO DE DADOS (CLEVR SIMPLIFICADO) ---
class ClevrSimplified:
    def __init__(self, num_samples=50):
        self.num_samples = num_samples
        self.data = self._generate()

    def _generate(self):
        data = []
        for _ in range(self.num_samples):
            pos = np.random.rand(2) # [0,1]
            c_idx = np.random.randint(0, 3)
            color = np.zeros(3); color[c_idx] = 1 # [2,3,4]
            s_idx = np.random.randint(0, 5)
            shape = np.zeros(5); shape[s_idx] = 1 # [5,6,7,8,9]
            is_large = np.random.rand() > 0.5
            size = np.array([1.0 if is_large else 0.0]) # [10]
            vec = np.concatenate([pos, color, shape, size])
            data.append(vec)
        return torch.tensor(np.array(data), dtype=torch.float32).to(device)

# --- 3. MODELOS NEURAIS ---
# Modelos para características (Shapes, Colors, Size)
class FeatureModel(nn.Module):
    def __init__(self, input_indices):
        super(FeatureModel, self).__init__()
        self.indices = input_indices
        input_dim = len(input_indices)
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16), nn.ELU(), nn.Linear(16, 1), nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x[..., self.indices])

# Modelos para Relações (Spatial)
class RelationModel(nn.Module):
    def __init__(self, axis_idx):
        super(RelationModel, self).__init__()
        self.axis = axis_idx
        self.net = nn.Sequential(
            nn.Linear(2, 16), nn.ELU(), nn.Linear(16, 1), nn.Sigmoid()
        )
    def forward(self, x, y):
        # Pega a coordenada específica (X=0 ou Y=1) de ambos os objetos
        return self.net(torch.cat([x[..., self.axis:self.axis+1], y[..., self.axis:self.axis+1]], dim=-1))

# Modelos Fixos (Matemáticos)
class CloseToModel(nn.Module):
    def forward(self, x, y):
        dist_sq = torch.sum((x[..., 0:2] - y[..., 0:2])**2, dim=-1, keepdim=True)
        return torch.exp(-2.0 * dist_sq)

class SameSizeModel(nn.Module):
    def forward(self, x, y):
        return 1.0 - torch.abs(x[..., 10:11] - y[..., 10:11])

# --- 4. FUNÇÃO PRINCIPAL DE EXECUÇÃO (Loop de 5x) ---
def run_experiment(run_id):
    print(f"\n--- INICIANDO EXECUÇÃO {run_id+1}/5 ---")

    # A. Instanciar Predicados (Reiniciando pesos a cada run)
    # Formas (indices 5 a 9)
    IsCircle = ltn.Predicate(FeatureModel([5]).to(device))
    IsSquare = ltn.Predicate(FeatureModel([6]).to(device))
    IsCylinder = ltn.Predicate(FeatureModel([7]).to(device))
    IsCone = ltn.Predicate(FeatureModel([8]).to(device))
    IsTriangle = ltn.Predicate(FeatureModel([9]).to(device))

    # Cores (2 a 4) e Tamanho (10)
    IsGreen = ltn.Predicate(FeatureModel([3]).to(device))
    IsSmall = ltn.Predicate(FeatureModel([10]).to(device)) # Assumindo 0=Small no treino, mas o feature é 0 ou 1
    # Nota: Se 0.0 é pequeno, a rede precisa aprender a inverter ou mapear 0->High.
    # Para facilitar, IsSmall pode ser Not(IsBig) ou treinado direto.

    # Relações
    LeftOf = ltn.Predicate(RelationModel(0).to(device)) # Eixo X
    RightOf = ltn.Predicate(RelationModel(0).to(device))
    Below = ltn.Predicate(RelationModel(1).to(device)) # Eixo Y
    Above = ltn.Predicate(RelationModel(1).to(device))

    CloseTo = ltn.Predicate(CloseToModel().to(device))
    SameSize = ltn.Predicate(SameSizeModel().to(device))

    # Operadores
    Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
    And = ltn.Connective(ltn.fuzzy_ops.AndProd())
    Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
    Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
    Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
    Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
    sat_agg = ltn.fuzzy_ops.AggregPMeanError(p=2)

    # B. Dados e Variáveis
    ds = ClevrSimplified(num_samples=60) # Um pouco mais de dados
    data = ds.data
    x = ltn.Variable("x", data)
    y = ltn.Variable("y", data)
    z = ltn.Variable("z", data)

    # C. Otimizador
    optimizer = torch.optim.Adam(
        list(IsCircle.parameters()) + list(IsSquare.parameters()) +
        list(IsCylinder.parameters()) + list(IsCone.parameters()) + list(IsTriangle.parameters()) +
        list(IsSmall.parameters()) + list(IsGreen.parameters()) +
        list(LeftOf.parameters()) + list(RightOf.parameters()) +
        list(Below.parameters()) + list(Above.parameters()), lr=0.01
    )

    # D. Loop de Treino
    print("Treinando...")
    for epoch in range(300): # 300 épocas é suficiente para esse problema simples
        optimizer.zero_grad()

        # --- AXIOMAS ---

        # 1. Taxonomia Completa (Um objeto deve ser uma e apenas uma forma)
        # Cobertura: É Círculo OU Quadrado OU Cilindro OU Cone OU Triângulo
        sat_cov = Forall(x, Or(Or(Or(Or(IsCircle(x), IsSquare(x)), IsCylinder(x)), IsCone(x)), IsTriangle(x)))

        # Exclusão Mútua (Exemplo par a par para simplificar código, ou lógica geral)
        # Forma simplificada: Se é Círculo, não pode ser Quadrado, etc.
        sat_mut_1 = Forall(x, Implies(IsCircle(x), Not(IsSquare(x))))
        sat_mut_2 = Forall(x, Implies(IsSquare(x), Not(IsCylinder(x)))) # Adicionar mais pares idealmente

        # 2. Espacial (LeftOf/RightOf)
        sat_lr_irref = Forall(x, Not(LeftOf(x, x)))
        sat_lr_inv = Forall([x, y], Implies(LeftOf(x, y), RightOf(y, x)))
        sat_lr_trans = Forall([x,y,z], Implies(And(LeftOf(x,y), LeftOf(y,z)), LeftOf(x,z))) # TRANSITIVIDADE

        # 3. Vertical (Below/Above)
        sat_ud_inv = Forall([x, y], Implies(Below(x, y), Above(y, x)))
        sat_ud_trans = Forall([x,y,z], Implies(And(Below(x,y), Below(y,z)), Below(x,z))) # TRANSITIVIDADE

        # 4. Supervisão (Grounding Simples para ancorar os conceitos)
        # Usamos máscaras reais do dataset para ensinar o que é o que
        mask_circ = data[:, 5] == 1
        mask_sq = data[:, 6] == 1
        mask_cyl = data[:, 7] == 1
        mask_cone = data[:, 8] == 1
        mask_tri = data[:, 9] == 1

        # Se houver dados dessa classe, ensina. Senão, ignora (Constant 1.0)
        s_circ = Forall(ltn.Variable("xc", data[mask_circ]), IsCircle(ltn.Variable("xc", data[mask_circ]))) if mask_circ.any() else ltn.Constant(1.)
        s_sq = Forall(ltn.Variable("xs", data[mask_sq]), IsSquare(ltn.Variable("xs", data[mask_sq]))) if mask_sq.any() else ltn.Constant(1.)
        s_cyl = Forall(ltn.Variable("xcy", data[mask_cyl]), IsCylinder(ltn.Variable("xcy", data[mask_cyl]))) if mask_cyl.any() else ltn.Constant(1.)
        s_cone = Forall(ltn.Variable("xco", data[mask_cone]), IsCone(ltn.Variable("xco", data[mask_cone]))) if mask_cone.any() else ltn.Constant(1.)
        s_tri = Forall(ltn.Variable("xt", data[mask_tri]), IsTriangle(ltn.Variable("xt", data[mask_tri]))) if mask_tri.any() else ltn.Constant(1.)

        # Agregação da Loss
        sat_total = sat_agg(torch.stack([
            sat_cov.value, sat_mut_1.value, sat_mut_2.value,
            sat_lr_irref.value, sat_lr_inv.value, sat_lr_trans.value,
            sat_ud_inv.value, sat_ud_trans.value,
            s_circ.value, s_sq.value, s_cyl.value, s_cone.value, s_tri.value
        ]))

        loss = 1.0 - sat_total
        loss.backward()
        optimizer.step()

    print(f"Loss Final Run {run_id+1}: {loss.item():.4f}")

    # --- 5. AVALIAÇÃO E CONSULTAS ---

    # A. Predicado 'canStack' (Tarefa 3)
    # Regra: Pode empilhar se Y NÃO for Cone E NÃO for Triângulo
    # Aqui definimos como uma fórmula composta, não predicado treinado, pois deriva de formas
    def canStack(obj_x, obj_y):
        # obj_x sobre obj_y. Regra depende de Y.
        return And(Not(IsCone(obj_y)), Not(IsTriangle(obj_y)))

    # B. Consultas Complexas (Tarefa 4)
    # Q1: Objeto Pequeno, Abaixo de Cilindro e Esq de Quadrado
    q1 = Exists(x, And(IsSmall(x), And(Exists(y, And(IsCylinder(y), Below(x,y))), Exists(z, And(IsSquare(z), LeftOf(x,z))))))

    # Q2: Cone Verde Entre dois objetos
    # InBetween(x,y,z) = (LeftOf(y,x) & RightOf(z,x)) OR ... Simplificado para LeftOf(y,x) & LeftOf(x,z)
    q2 = Exists([x,y,z], And(And(IsCone(x), IsGreen(x)), And(LeftOf(y,x), LeftOf(x,z))))

    # C. Cálculo de Métricas (Shape Classification)
    true_shapes = torch.argmax(data[:, 5:10], dim=1).cpu().numpy()

    # Forward em todos os predicados de forma
    p_s = []
    for pred in [IsCircle, IsSquare, IsCylinder, IsCone, IsTriangle]:
        p_s.append(pred.model(data).detach().cpu().numpy())
    pred_shapes = np.argmax(np.stack(p_s, axis=1).squeeze(), axis=1)

    acc = accuracy_score(true_shapes, pred_shapes)
    prec = precision_score(true_shapes, pred_shapes, average='macro', zero_division=0)
    rec = recall_score(true_shapes, pred_shapes, average='macro', zero_division=0)
    f1 = f1_score(true_shapes, pred_shapes, average='macro', zero_division=0)

    # Retorna dicionário de resultados
    return {
        "sat_q1": q1.value.item(),
        "sat_q2": q2.value.item(),
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }

# --- 6. EXECUÇÃO DOS 5 EXPERIMENTOS ---
results = []
for i in range(5):
    res = run_experiment(i)
    results.append(res)

# --- 7. RELATÓRIO FINAL ---
print("\n=== RESULTADOS FINAIS (MÉDIA DE 5 EXECUÇÕES) ===")
keys = results[0].keys()
for k in keys:
    values = [r[k] for r in results]
    mean_val = np.mean(values)
    std_val = np.std(values)
    print(f"{k}: {mean_val:.4f} (+/- {std_val:.4f})")

Rodando em: cpu

--- INICIANDO EXECUÇÃO 1/5 ---
Treinando...
Loss Final Run 1: 0.0008

--- INICIANDO EXECUÇÃO 2/5 ---
Treinando...
Loss Final Run 2: 0.0009

--- INICIANDO EXECUÇÃO 3/5 ---
Treinando...
Loss Final Run 3: 0.0007

--- INICIANDO EXECUÇÃO 4/5 ---
Treinando...
Loss Final Run 4: 0.0008

--- INICIANDO EXECUÇÃO 5/5 ---
Treinando...
Loss Final Run 5: 0.0007

=== RESULTADOS FINAIS (MÉDIA DE 5 EXECUÇÕES) ===
sat_q1: 0.0001 (+/- 0.0000)
sat_q2: 0.0001 (+/- 0.0000)
accuracy: 1.0000 (+/- 0.0000)
precision: 1.0000 (+/- 0.0000)
recall: 1.0000 (+/- 0.0000)
f1: 1.0000 (+/- 0.0000)
